# Rectangular data loader

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#export
from nb_003a import *

In [ ]:
DATA_PATH = Path('data')
PATH = DATA_PATH/'caltech101'

np.random.seed(42)
train_ds,valid_ds = FilesDataset.from_folder(PATH, test_pct=0.2)

x = train_ds[-1][0]
classes = train_ds.classes
c = len(classes)

len(train_ds),len(valid_ds),c

## Closest ntile

In [ ]:
show_image(train_ds[1][0], figsize=(6,3))
x.shape

In [ ]:
asp_ratios = [operator.truediv(*Image.open(fn).size) for fn in train_ds.fns]
asp_ratios[:4]

In [ ]:
asp_ntiles = np.percentile(asp_ratios, [2,20,50,80,98])
asp_ntiles

In [ ]:
def closest_ntile(aspect, ntiles):
    return ntiles[np.argmin(abs(log(aspect)-log(ntiles)))]

In [ ]:
aspect = x.shape[2]/x.shape[1]
nearest_aspect = closest_ntile(aspect, asp_ntiles)
aspect,nearest_aspect

In [ ]:
get_crop_target(128, nearest_aspect)

## SortAspectBatchSampler

In [ ]:
asp_nearests = [closest_ntile(o, asp_ntiles) for o in asp_ratios]
asp_nearests[:10]

In [ ]:
bs=32

In [ ]:
from itertools import groupby

In [ ]:
sort_nearest = sorted(enumerate(asp_nearests), key=itemgetter(1))
groups = [list(b) for a,b in groupby(sort_nearest, key=itemgetter(1))]
len(groups)

In [ ]:
groups[0][:4]

In [ ]:
sum(math.ceil(len(g)/bs) for g in groups)

In [ ]:
@dataclass
class SortAspectBatchSampler(Sampler):
    ds:Dataset; bs:int; shuffle:bool = False

    def __post_init__(self):
        asp_ratios = [operator.truediv(*Image.open(img).size) for img in self.ds.fns]
        asp_ntiles = np.percentile(asp_ratios, [2,20,50,80,98])
        asp_nearests = [closest_ntile(o, asp_ntiles) for o in asp_ratios]
        sort_nearest = sorted(enumerate(asp_nearests), key=itemgetter(1))
        self.groups = [list(b) for a,b in groupby(sort_nearest, key=itemgetter(1))]
        self.n = sum(math.ceil(len(g)/self.bs) for g in self.groups)

    def __len__(self): return self.n
    
    def __iter__(self):
        if self.shuffle: self.groups = sample(self.groups, len(self.groups))
        for group in self.groups:
            group = [(a,{'aspect':b}) for a,b in group]
            if self.shuffle: group = sample(group, len(group))
            for i in range(0, len(group), self.bs): yield group[i:i+self.bs]

In [ ]:
next(iter(SortAspectBatchSampler(train_ds, 4)))

In [ ]:
it = iter(SortAspectBatchSampler(train_ds, 4, True))
next(it),next(it)

## Rectangular dataset

In [ ]:
class TfmDataset(Dataset):
    def __init__(self, ds: Dataset, tfms: Collection[Callable] = None, **kwargs):
        self.ds,self.tfms,self.kwargs = ds,tfms,kwargs
        
    def __len__(self): return len(self.ds)
    def __getattr__(self, k): return getattr(self.ds, k)
#     def fns(self): return self.ds.fns
    
    def __getitem__(self,idx):
        if isinstance(idx, tuple): idx,xtra = idx
        else: xtra={}
        x,y = self.ds[idx]
        return apply_tfms(self.tfms)(x, **self.kwargs, **xtra), y

In [ ]:
tfms = [
    rotate_tfm(degrees=(-20,20.)),
    zoom_tfm(scale=(1.,2.))
]

train_tds = TfmDataset(train_ds, tfms)

In [ ]:
_,axes = plt.subplots(2,2, figsize=(8,6))
for ax in axes.flat: show_image(train_tds[1][0], ax, hide_axis=False)

In [ ]:
train_tfms = [
    rotate_tfm(degrees=(-20,20.)),
    zoom_tfm(scale=(1.,2.), row_pct=(0,1.), col_pct=(0,1.)),
    crop_pad_tfm(row_pct=(0,1.), col_pct=(0,1.))
]
valid_tfms = [
    zoom_tfm(),
    crop_pad_tfm()
]

In [ ]:
train_tds = TfmDataset(train_ds, train_tfms)
valid_tds = TfmDataset(valid_ds, valid_tfms)

xtra = {'size':100}
train_tds[(1,xtra)][0].shape

In [ ]:
_,axes = plt.subplots(2,2, figsize=(8,6))
for i,ax in enumerate(axes.flat):
    im = train_tds[(i, xtra)][0]
    print(im.shape)
    show_image(im, ax, hide_axis=False)

In [ ]:
class DataBunch():
    def __init__(self, train_dl, valid_dl, device=None):
        self.device = default_device if device is None else device
        self.train_dl = DeviceDataLoader(train_dl, device=self.device)
        self.valid_dl = DeviceDataLoader(valid_dl, device=self.device)

    @classmethod
    def create(cls, train_ds, valid_ds, bs=64, device=None, num_workers=4,
               train_tfm=None, valid_tfm=None, sample_func=None):
        if train_tfm is not None: train_tfm = TfmDataset(train_ds, train_tfm)
        if valid_tfm is not None: valid_tfm = TfmDataset(valid_ds, valid_tfm)
        if sample_func is None:
            train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=num_workers)
            valid_dl = DataLoader(valid_ds, bs*2, shuffle=False, num_workers=num_workers)
        else:
            train_samp = sample_func(train_ds, bs, True)
            valid_samp = sample_func(valid_ds, bs*2, False)
            train_dl = DataLoader(train_ds, num_workers=num_workers, batch_sampler=train_samp)
            valid_dl = DataLoader(valid_ds, num_workers=num_workers, batch_sampler=valid_samp)
        return cls(train_dl, valid_dl, device)
        
    @property
    def train_ds(self): return self.train_dl.dl.dataset
    @property
    def valid_ds(self): return self.valid_dl.dl.dataset

In [ ]:
train_tds = TfmDataset(train_ds, train_tfms, size=100, padding_mode='zeros')
valid_tds = TfmDataset(valid_ds, valid_tfms, size=100, padding_mode='zeros')

In [ ]:
len(train_tds), len(valid_tds)

In [ ]:
_,axes = plt.subplots(2,2, figsize=(8,6))
for i,ax in enumerate(axes.flat):
    im = train_tds[(i)][0]
    print(im.shape)
    show_image(im, ax, hide_axis=False)

In [ ]:
data = DataBunch.create(train_tds, valid_tds, bs, num_workers=0)

In [ ]:
x,y = next(iter(data.train_dl))

_,axes = plt.subplots(2,4, figsize=(9,3))
for i,ax in enumerate(axes.flat): show_image(x[i], ax)

In [ ]:
data = DataBunch.create(train_tds, valid_tds, bs, sample_func=SortAspectBatchSampler, num_workers=0)

In [ ]:
print(x[0].shape)
_,axes = plt.subplots(2,4, figsize=(12,4))
for i,ax in enumerate(axes.flat): show_image(x[i], ax, hide_axis=False)

## fin

Alternative sampler

In [ ]:
@dataclass
class SortAspectBatchSampler(Sampler):
    ds:Dataset; bs:int; shuffle:bool = False

    def __post_init__(self):
        asp_ratios = [operator.truediv(*Image.open(img).size) for img in self.ds.fns]
        asp_ntiles = np.percentile(asp_ratios, [2,20,50,80,98])
        asp_nearests = [closest_ntile(o, asp_ntiles) for o in asp_ratios]
        sort_nearest = sorted(enumerate(asp_nearests), key=itemgetter(1))
        self.groups = [list(b) for a,b in groupby(sort_nearest, key=itemgetter(1))]
        self.n = sum(math.ceil(len(g)/self.bs) for g in self.groups)

    def __len__(self): return self.n
    
    def __iter__(self):
        groups1,batch_idx = [],[]
        for i, group in enumerate(self.groups):
            group = [(a,{'aspect':b}) for a,b in group]
            if self.shuffle: group = sample(group, len(group))
            groups1.append(group)
            batch_idx += [(i,j) for j in range(0,len(group),self.bs)]
        if self.shuffle: batch_idx = sample(batch_idx, len(batch_idx))
        for idx in batch_idx:
            yield groups1[idx[0]][idx[1]:idx[1]+self.bs]